<a href="https://colab.research.google.com/github/KhresnaPanduI/ASL-Model/blob/main/asl_model_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [1]:
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub
from keras.preprocessing.image import ImageDataGenerator

from tqdm import tqdm


In [2]:
module_selection = ("mobilenet_v2", 224, 1280) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and output dimension {}".format(MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280


Data extraction

In [3]:
train_dir = '/content/drive/MyDrive/Datasets/own asl/Train'
test_dir = '/content/drive/MyDrive/Datasets/own asl/test'

Number of class

In [4]:
import os

categories = os.listdir(train_dir)
categoriest = os.listdir(test_dir)
print(len(categories))
print(len(categoriest))

29
29


In [5]:
train_data_gen = ImageDataGenerator(
                              rescale=1/255)

test_data_gen = ImageDataGenerator(
                              rescale=1/255)

In [6]:
train_gen = train_data_gen.flow_from_directory(train_dir,
                                         target_size=(224,224),
                                         class_mode="categorical",
                                         color_mode="rgb",
                                         shuffle=True,
                                         batch_size=32)

val_gen = test_data_gen.flow_from_directory(test_dir,
                                          target_size=(224,224),
                                          class_mode="categorical",
                                          color_mode="rgb",
                                          shuffle=True,
                                          batch_size=32,)

Found 1171 images belonging to 29 classes.
Found 289 images belonging to 29 classes.


In [7]:
feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,), 
                                   output_shape=[FV_SIZE],
                                   trainable=True)

In [8]:
print("Building model with", MODULE_HANDLE)

model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(29, activation='softmax')
])

model.summary()

Building model with https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 29)                37149     
Total params: 2,295,133
Trainable params: 2,261,021
Non-trainable params: 34,112
_________________________________________________________________


Unfreeze some layer

In [9]:
NUM_LAYERS = 10 #@param {type:"slider", min:1, max:50, step:1}
      
feature_extractor.trainable = True
    
for layer in model.layers[-NUM_LAYERS:]:
  layer.trainable = True

Training the model

In [10]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])

hist = model.fit(train_gen,
                 epochs = 10,
                 validation_data = val_gen)

Epoch 1/10
37/37 [==============================] - 462s 11s/step - loss: 1.5193 - accuracy: 0.6450 - val_loss: 2.8999 - val_accuracy: 0.3979
Epoch 2/10
37/37 [==============================] - 9s 233ms/step - loss: 0.2244 - accuracy: 0.9650 - val_loss: 9.0491 - val_accuracy: 0.1730
Epoch 3/10
37/37 [==============================] - 9s 233ms/step - loss: 0.3457 - accuracy: 0.9255 - val_loss: 2.3735 - val_accuracy: 0.5467
Epoch 4/10
37/37 [==============================] - 9s 233ms/step - loss: 0.1571 - accuracy: 0.9861 - val_loss: 1.1629 - val_accuracy: 0.6955
Epoch 5/10
37/37 [==============================] - 9s 235ms/step - loss: 0.1268 - accuracy: 0.9933 - val_loss: 1.0543 - val_accuracy: 0.7163
Epoch 6/10
37/37 [==============================] - 9s 235ms/step - loss: 0.1416 - accuracy: 0.9871 - val_loss: 6.9471 - val_accuracy: 0.2837
Epoch 7/10
37/37 [==============================] - 9s 239ms/step - loss: 0.1363 - accuracy: 0.9789 - val_loss: 0.9253 - val_accuracy: 0.8097
Epoch 

In [11]:
ASL_SAVED_MODEL = "asl_saved_model"

In [12]:
tf.saved_model.save(model, ASL_SAVED_MODEL)

INFO:tensorflow:Assets written to: asl_saved_model/assets


INFO:tensorflow:Assets written to: asl_saved_model/assets


In [13]:
%%bash -s $ASL_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['keras_layer_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: serving_default_keras_layer_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 29)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [14]:
def representative_dataset_gen():
  for i in range(50):
    image = tf.random.uniform([1, 224, 224, 3])
    yield [image]

imported = tf.saved_model.load(ASL_SAVED_MODEL)

concrete_func = imported.signatures["serving_default"]

concrete_func.inputs[0].set_shape([1, 224, 224, 3])

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])

converter.optimizations = [tf.lite.Optimize.DEFAULT] 

converter.representative_dataset = representative_dataset_gen

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.inference_input_type = tf.uint8

converter.inference_output_type = tf.uint8

tflite_model = converter.convert() 

In [15]:
tflite_model_file = 'quantized_asl_model.tflite'

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

Create file to save labels

In [16]:
class_names = ['a', 'b', 'c', 'd', 'e', 'del', 'f', 'g', 'h',
               'i', 'j', 'k', 'l', 'm', 'n', 'nothing', 'o', 'p',
               'q', 'r', 's', 'space', 't', 'u', 'v', 'w', 'x', 
               'y', 'z']

with open('labels.txt', 'w') as f:
    f.write('\n'.join(class_names))

In [18]:
try:
    from google.colab import files
    files.download('quantized_asl_model.tflite')
    files.download('labels.txt')
except:
    pass



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>